In [12]:
import mph
import os
import time
import config
import pymph
import numpy as np
import matplotlib.pyplot as plt

In [3]:
main.build()

In [13]:
config.model.study("std1").run()

In [17]:
config.model.result()

config.model.evaluationGroup("eg2").feature("gmev1")
config.model.result().evaluationGroup("eg2").run();

com.comsol.util.exceptions.FlException: Exception:
	com.comsol.util.exceptions.FlException: Unknown node group
Messages:
	Unknown node group.
	- Tag: grp1
